In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import glob
import tarfile
from Bio import SeqIO
from scipy import stats
import statsmodels.formula.api as smf

from matplotlib import pyplot as plt

# Making use of Representative Genomes to limit species to include in OMA orthologs

See: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0018910

This step was performed manually, using the "Export marker genes" functionality of OMA (Jan. 2021, see: https://omabrowser.org/oma/export_markers).

Briefly, I selected three datasets: 

1. From *all* gammaproteobacteria, I used the rgg-15 database
2. From Enterobacterales I used the rp75 database
3. From Enterobacteriacaea I used the rp75 database

In each case, I attempted to manually include every genome listed in the associated rgg database below the indicated taxonomy level. This wasn't always possible due to some database discrepancies but resulted in fairly broad coverage.

In each case I exported all marker genes (Maximum nr of markers = -1) and ensured that each ortholog occured in 50% of coverd species (Minimum fraction of covered species: 0.5)

In [ ]:
###CHOOSE ONE

###Went through and tried to find all of the genomes present from this set
# df = pd.read_csv('../Data/ecoli_info/evolutionary_analysis/rgg-15.txt', sep='\t',
#                  encoding = 'ISO-8859-1', header=None)




##For this set I used OMA to first limit taxonomy then for each genus tried to find
##species in this dataframe (given that the numbers were so much larger and include
##lots of species outside of my planned for taxonomy)
# df = pd.read_csv('../Data/ecoli_info/evolutionary_analysis/rgg-75.txt', sep='\t',
#                  encoding = 'ISO-8859-1', header=None)





# print(df.shape)
# df = df[df[0].str[0]=='>']
# print(df.shape)
# df = df[df[8] == 'Bac/Gamma-proteo']
# print(df.shape)
# df = df[df[6] != '9GAMM']
# print(df.shape)
# df.head()

**Manually looking for certain names in the rgg database, used when finding species to include in the Enterobacterales and Enterobactericaea clades**

In [ ]:
# for i in df.index:
#     if 'Escherichia' in df.loc[i][4]:
#         print(df.loc[i][4], '*****', df.loc[i][6])
# #     print(df.loc[i][4], '*****', df.loc[i][6])

# Selecting group and species sets to move forward with

Parse through the downloaded orthologs and try to find a more limited set of: 

1. species and 
2. ortholog families

to move forward with that would ensure relatively similar trees.

This code won't run right now because I've chosen to zip up the files into a `.tar.gz` file. So this code was run once for each dataset and should be considered "complete"

In [ ]:
# from collections import Counter, defaultdict
# import gzip
# import subprocess

In [ ]:
# # data_loc = '../Data/ecoli_info/evolutionary_analysis/marker_genes_rp15_gammaproteobacteria/'
# # data_loc = '../Data/ecoli_info/evolutionary_analysis/marker_genes_rp75_enterobacterales/'
# data_loc = '../Data/ecoli_info/evolutionary_analysis/marker_genes_rp75_enterobacteriacaea/'
# # data_loc = '../Data/ecoli_info/evolutionary_analysis/marker_genes_rp75_escherichia/'

# fasta_files = glob.glob(data_loc + '*.fa')
# print(len(fasta_files))

**First getting the number of genes per genome, and removing genomes with too few hits**

In [ ]:
# ###Dictionary to hold hit counts per species
# species_count_dict = defaultdict(int)

# for fasta_file in fasta_files[:]:
#     records = list(SeqIO.parse(fasta_file, 'fasta'))
#     ids = [record.id[:5] for record in records]
#     id_counter = Counter(ids)
#     for single_id in id_counter:
#         species_count_dict[single_id]+=1
        
# temp_items = species_count_dict.items()
# listy = sorted(temp_items, key=lambda x: x[1])
# max_val = listy[-1][1] ###Identify the largest number of hits
# to_delete = [i[0] for i in listy if i[1] < max_val*0.8] ###Remove species below this threshold
# print('Species to remove:', to_delete)
# valid_species = [i[0] for i in listy if i[0] not in to_delete]
# valid_species_count = len(valid_species)
# print('Number of species to keep:', valid_species_count)

**Given the newly limited set of species, identify high coverage ortholog families (i.e. those with representatives in most genomes**

In [ ]:
# ###What I ultimately wish to keep
# valid_fasta_files = []

# ###Dictionary to hold hit counts per genome set
# og_count_dict = defaultdict(int)
# for fasta_file in fasta_files[:]:
#     records = list(SeqIO.parse(fasta_file, 'fasta'))
#     ids = [record.id[:5] for record in records]
    
#     ###Select only from the valid species list
#     ids = sorted([single_id for single_id in ids if single_id not in to_delete])
#     if len(ids) < valid_species_count * 0.8: ###Remove orthologs below this threshold
#         continue
#     valid_fasta_files.append(fasta_file)
    
#     ###Putting together the genome set for the ortholog
#     long_id = '_%_'.join(ids)
#     og_count_dict[long_id] += 1
    
# print(np.sum(list(og_count_dict.values())))
# print(len(valid_fasta_files))

**Inspecting the genome set hits (solely as a check / out of curiosity)**

In [ ]:
# temp_items = og_count_dict.items()
# listy = sorted(temp_items, key=lambda x: x[1], reverse=True)
# for i in listy[:5]:
#     print(i)
#     print()

**Write \*new\* ortholog fasta files for the ortholog families that passed the test and limit these files to include only the valid set of genomes. Place in a new folder**

In [ ]:
# for fasta_file in valid_fasta_files:
#     print(fasta_file)
#     records = list(SeqIO.parse(fasta_file, 'fasta'))
#     records = [record for record in records if record.id[:5] in valid_species]
#     with open(fasta_file.replace('marker_genes_', ''), 'w') as outfile:
#         SeqIO.write(records, outfile, 'fasta')

**Make sure that I have the coding sequences that I need on hand. These are going to be extracted from the humongous OMA groups cdna file for the species that I care about**

In [ ]:
# out_dir = '../Data/ecoli_info/evolutionary_analysis/CODING_SEQUENCES/'

# ###Make sure that I don't repeat species since this takes some time to run
# finished_species = [i.split('/')[-1][:-10] for i in glob.glob(out_dir+'*')]
# print('Finished species:\n', finished_species)
# print()
# species_to_write = [i for i in valid_species if i not in finished_species]
# print('New species to write:\n', species_to_write)

**Slowly iterate through the huge cDNA file and for every record see if it's in the species list that I care about, if so... save it in a dictionary and at the end write each new file**

In [ ]:
# if species_to_write != []:
#     cds_dicty = defaultdict(dict)
#     with gzip.open('../Data/ecoli_info/evolutionary_analysis/prokaryotes.cdna.fa.gz', 'rt') as infile:
#         for record in SeqIO.parse(infile, 'fasta'):
#             if record.id[:5] not in species_to_write:
#                 continue
#             cds_dicty[record.id[:5]][record.id] = record
#     for key, seq_dict in cds_dicty.items():
#         fname = '../Data/ecoli_info/evolutionary_analysis/CODING_SEQUENCES/{}.cds.fasta'.format(key)
#         with open(fname, 'w') as outfile:
#             SeqIO.write(list(seq_dict.values()), outfile, 'fasta')

# Create corresponding CDS files for each OMA aa file

Similar to above, I've subsequently zipped all these files up so this code won't currently run but was how I compiled everything.

In [ ]:
# all_cds_seqs = {}
# for genome_file in glob.glob('../Data/ecoli_info/evolutionary_analysis/CODING_SEQUENCES/*.cds.fasta'):
#     records = SeqIO.parse(genome_file, 'fasta')
#     for record in records:
#         all_cds_seqs[record.id] = record
# print(len(all_cds_seqs.keys()))

In [ ]:
# good_files = glob.glob(data_loc.replace('marker_genes_', '')+'*.fa')
# print('Number of amino acid fasta files to work with:', len(good_files))
# for fa_file in good_files[:]:
#     aa_records = list(SeqIO.parse(fa_file, 'fasta'))
#     cds_records = [all_cds_seqs[record.id] for record in aa_records]
#     assert len(aa_records) == len(cds_records) ###Make sure no errors popped up
#     with open(fa_file.replace('.fa', '.cds.fna'), 'w') as outfile:
#         for i, record in enumerate(aa_records):
#             outfile.write('>{}\n'.format(record.description))
#             outfile.write('{}\n'.format(str(cds_records[i].seq)))

# Run MAFFT to align amino acid sequences

Just doing a basic MAFFT analysis, could consider some more intense parameters at some point

In [ ]:
# for fa_file in good_files:
#     aligned_file = fa_file.replace('.fa', '.aln')
#     subprocess.call('mafft {} > {}'.format(fa_file, aligned_file), shell=True)

# Get nucleotide alignments from aligned amino acid sequences and raw nucleotide sequences using `pal2nal`.

I thought about doing this manually before, but seems like this program is pretty standard?

In [ ]:
# print(len(good_files))
# for fa_file in good_files[:]:
#     aln_file = fa_file.replace('.fa', '.aln')
#     cds_file = fa_file.replace('.fa', '.cds.fna')
#     cds_align = cds_file.replace('.fna', '.aln')
#     with open(cds_align, 'w') as outfile:
#         subprocess.call('~/workspace/pal2nal/pal2nal.pl -output fasta -nostderr {} {}'.format(aln_file, cds_file),
#                         stdout=outfile,
#                         shell=True)

**Remove X's so that codeml actually runs**

In [ ]:
# for fna_file in glob.glob(data_loc.replace('marker_genes_', '') + '*.cds.aln'):
#     new_ids = []
#     new_seqs = []
#     records = SeqIO.parse(fna_file, 'fasta')
#     for record in records:
#         new_ids.append(record.id)
#         if 'X' in str(record.seq):
#             new_seqs.append(str(record.seq).replace('X', '-'))
#         else:
#             new_seqs.append(str(record.seq))
#     with open(fna_file, 'w') as outfile:
#         for i,j in zip(new_ids, new_seqs):
#             outfile.write('>{}\n{}\n'.format(i, j))

# Run FastTree to make some trees from the amino acid sequences

Note the `-nosupport` flag.

I think it's better to make / use amino acid trees rather than the nucleotide level trees. Also note that I construct gene trees rather than doing a concatenated analysis and creating a large species tree (which is something that I could consider given the large number of orthologs that appear in all genomes that I selected, regardless of dataset). 

In [ ]:
# for fa_file in good_files[:]:
#     print(fa_file)
#     aln_file = fa_file.replace('.fa', '.aln')
#     tree_file = fa_file.replace('.fa', '.newick')
#     with open(tree_file, 'w') as outfile:
#         subprocess.call('~/workspace/FastTree/FastTree -nosupport -lg {}'.format(aln_file),
#                         stdout=outfile,
#                         shell=True)

# Offline: run programs to get evolutionary rates!

This is the payoff but also a real treat, given that both programs are astonishingly opaque and difficult to work with. I separately considered both `hyphy` and `codeml` (from `paml`). 

For `hyphy`, this is a pretty nice resource: https://stevenweaver.github.io/hyphy-site/tutorials/current-release-tutorial/ but note that the actual terminal level commands end up looking like this:
`(echo 5; echo 1; echo 1; echo /ABSOLUTE/PATH/TO/ALIGNMENT.CDS.ALN; echo MG94CUSTOMCF3X4; echo 2; echo 012345; echo /ABSOLUTE/PATH/TO/TREE.NEWICK; echo 1) | hyphy | tail -n 11 | head -n 9 > output.txt`

All of the `echo`-ing is to avoid the interactive interface, and the `tail` to `head` to output is all just to get a simple and clean output file that I can parse. 

For the purposes of this project I ran all of this code on our local cluster using include `.py` scripts in the folder `cluster_scripts/`.

# Read all dn_ds files and construct relevant columns

Trying to test here how reliable these values are, which program seems best, and which dataset is most appropriate in terms of the scope of studying evolutionary rate.

## First trying to get a mapping of OMA groups to *E. coli* "bxxxx numbers"

In [ ]:
master_df = pd.read_csv('../Data/ecoli_info/current_ecoli_master_table.tsv', sep='\t')
master_df.shape
master_df.head()

,locus_tag,gene,start_loc,stop_loc,strand,cds_seq,us_seq,cds_len,well_behaved,GC_percent_cds,roc_semppr_mean,iCUB,CAI,tAI,stAIcalc
0,b0001,thrL,189,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,CAGATAAAAATTACAGAGTACACAACATCC,66,True,0.515152,1.244106,32.046035,0.617266,0.262286,0.258417
1,b0002,thrA,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,2463,True,0.530654,1.034078,55.949262,0.353246,0.230564,0.216862
2,b0003,thrB,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,933,True,0.562701,0.994168,56.062386,0.357812,0.216292,0.210420
3,b0004,thrC,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,1287,True,0.528361,1.176750,53.052776,0.394675,0.231407,0.209784
4,b0005,yaaX,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,297,True,0.538721,0.837528,50.705530,0.374371,0.197715,0.208944


In [ ]:
ecoli_seq_dict = {}     

# base_dirs = ['../Data/ecoli_info/evolutionary_analysis/rp15_gammaproteobacteria',
#              '../Data/ecoli_info/evolutionary_analysis/rp75_enterobacterales',
#              '../Data/ecoli_info/evolutionary_analysis/rp75_enterobacteriacaea',
#              '../Data/ecoli_info/evolutionary_analysis/rp75_escherichia']
base_dirs = ['../Data/ecoli_info/evolutionary_analysis/rp75_enterobacterales',
             '../Data/ecoli_info/evolutionary_analysis/rp75_enterobacteriacaea']

for base_dir in base_dirs:
    tar_fname = base_dir + '/OMAGroup_cds_fna_files.tar.gz'
    try:
        t = tarfile.open(tar_fname, 'r')
    except IOError as e:
        print(e)

    for infile in t.getmembers():
        oma_name = infile.name.split('/')[-1].split('.')[0]
        if oma_name in list(ecoli_seq_dict.keys()):
            continue
        records = t.extractfile(infile).read().decode("utf-8")
        records = records.split('\n')
        for i, record in enumerate(records):
            if record[:6] == '>ECOLI':
                ecoli_seq_dict[oma_name] = records[i+1].replace('-','')
    print(len(ecoli_seq_dict.keys()))
    t.close()

1982
2372


In [5]:
ecoli_oma_to_bnumber = {}
for name, seq in ecoli_seq_dict.items():
    temp = master_df[master_df['cds_seq'] == seq]
    if temp.shape[0] == 1:
        ecoli_oma_to_bnumber[name] = temp.iloc[0]['locus_tag']
print(len(ecoli_oma_to_bnumber.keys()))

2365


## Now extract `hyphy` results

As best as I can tell, the only real interesting number I can get from `hyphy` is a `dn/ds` type metric (denoted `R`) rather than separate measurements of `dn` and `ds`

**Instantiate an empty dataframe that will hold *all* rate results**

In [6]:
res_df = pd.DataFrame(list(ecoli_oma_to_bnumber.values()))
res_df.columns = ['locus_tag']

**Now add the various `hyphy` columns**

In [7]:
for base_dir in base_dirs:
    names = []
    rates = []
    tar_fname = base_dir + '/OMAGroup_hyphy_files.tar.gz'
    try:
        t = tarfile.open(tar_fname, 'r')
    except IOError as e:
        print(e)

    for infile in t.getmembers()[:]:
        oma_name = infile.name.split('/')[-1].split('.')[0]
        try:
            eco_name = ecoli_oma_to_bnumber[oma_name]
        except KeyError:
            continue
        df = pd.read_csv(t.extractfile(infile.name), skiprows=3, header=None, sep='=', index_col=0)
        rate = float(df.loc['R'][1].strip(';'))
        rates.append(rate)
        names.append(eco_name)
    temp_df = pd.DataFrame(zip(names, rates))
    temp_df.columns = ['locus_tag', 'hyphy_{}'.format(base_dir.split('/')[-1])]
    res_df = res_df.merge(temp_df, on='locus_tag', how='left')
    t.close()

In [8]:
print(res_df.shape)
res_df.head()

(2365, 3)


,locus_tag,hyphy_rp75_enterobacterales,hyphy_rp75_enterobacteriacaea
0,b3242,0.061857,0.041733
1,b3847,0.068649,0.044641
2,b3712,0.112357,0.107174
3,b2216,0.120750,0.101621
4,b1834,0.084874,0.064982


## Next read `codeml` outputs and construct rate column/s

This code runs really slow on the larger `.tar.gz` files, not sure what's going on but it completes. And I'm running it once or twice at best and saving the results. So can investigate why some other time. 

Note that for `codeml` I believe there are equivalent `dn`, `ds`, and `dn/ds` values to be extracted. I'm still taking alignment-wide values rather than trying to isolate the *E. coli* clade. 

In [9]:
for base_dir in base_dirs:
    names = []
    dn_vals = []
    ds_vals = []
    dn_ds_vals = []

    tar_fname = base_dir + '/OMAGroup_codeml_files.tar.gz'
    try:
        t = tarfile.open(tar_fname, 'r')
    except IOError as e:
        print(e)

    for infile in t.getmembers()[:]:
        oma_name = infile.name.split('/')[-1].split('.')[0]
        try:
            eco_name = ecoli_oma_to_bnumber[oma_name]
        except KeyError:
            continue
        lines = t.extractfile(infile.name).read().decode('utf-8')
        lines = lines.split('\n')
        for i, line in enumerate(lines):
            if 'dN & dS for each branch' == line:
                df = pd.DataFrame([temp_line.split() for temp_line in lines[i+4:-7]])
                df.columns = lines[i+2].split()
                names.append(eco_name)
                for col_name in ['dN', 'dS', 'dN/dS']:
                    df[col_name] = df[col_name].astype(float)
                dn_vals.append(df['dN'].mean())
                ds_vals.append(df['dS'].mean())
                dn_ds_vals.append(df['dN/dS'].mean())
                break
    temp_df = pd.DataFrame(zip(names, dn_vals, ds_vals, dn_ds_vals))
    temp_df.columns = ['locus_tag',
                      'codeml_dn_{}'.format(base_dir.split('/')[-1]),
                      'codeml_ds_{}'.format(base_dir.split('/')[-1]),
                      'codeml_dnds_{}'.format(base_dir.split('/')[-1])]
    res_df = res_df.merge(temp_df, on='locus_tag', how='left')
    t.close()
    print(res_df.shape)

(2365, 6)
(2365, 9)


In [10]:
print(res_df.shape)
res_df.head()

(2365, 9)


,locus_tag,hyphy_rp75_enterobacterales,hyphy_rp75_enterobacteriacaea,codeml_dn_rp75_enterobacterales,codeml_ds_rp75_enterobacterales,codeml_dnds_rp75_enterobacterales,codeml_dn_rp75_enterobacteriacaea,codeml_ds_rp75_enterobacteriacaea,codeml_dnds_rp75_enterobacteriacaea
0,b3242,0.061857,0.041733,0.019432,0.430200,0.0452,0.008914,0.254165,0.0351
1,b3847,0.068649,0.044641,0.018641,0.286941,0.0649,0.007812,0.193456,0.0404
2,b3712,0.112357,0.107174,0.062285,0.573070,0.1087,0.037463,0.365068,0.1026
3,b2216,0.120750,0.101621,0.044529,0.416921,0.1068,0.021724,0.235298,0.0923
4,b1834,0.084874,0.064982,0.027663,0.341353,0.0810,0.015866,0.251659,0.0630


In [11]:
assert len(set(res_df.index)) == len(list(res_df.index))

In [12]:
res_df.to_csv('../Data/ecoli_info/ecoli_evolutionary_rate_table.tsv', sep='\t', index=False)